<a href="https://colab.research.google.com/github/yimstar9/Engineer-Big-Data-Analysis_R/blob/main/%5B%EC%9D%B4%EA%B8%B0%EC%A0%81%EC%B9%B4%ED%8E%98_1%EC%A3%BC%EC%B0%A8_%EC%98%88%EC%83%81%EB%AC%B8%EC%A0%9C2%5D_%EC%9C%A0%EB%B0%A9%EC%95%94_%EB%B0%9C%EC%83%9D%EC%97%AC%EB%B6%80_%EC%98%88%EC%B8%A1_(%EC%A2%85%EC%86%8D%EB%B3%80%EC%88%98_diagnosis_B(%EC%96%91%EC%84%B1)_%2C_M(%EC%95%85%EC%84%B1)_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 데이터 출처 : https://archive.ics.uci.edu/ml/datasets/Breast%20Cancer%20Wisconsin%20(Diagnostic)에서 변형
# 데이터 설명 :  유방암 발생여부 예측 (종속변수 diagnosis : B(양성)  , M(악성) )
# 문제타입 : 분류유형
# 평가지표 : f1-score
# trainData url : https://raw.githubusercontent.com/Datamanim/datarepo/main/cancer/train.csv
# testData url : https://raw.githubusercontent.com/Datamanim/datarepo/main/cancer/test.csv
# subData url : https://raw.githubusercontent.com/Datamanim/datarepo/main/cancer/submission.csv

library(dplyr)
library(caret)
library(car)
library(lmtest)
library(ROCR)
library(ModelMetrics)
library(randomForest)
library(rpart)
library(e1071)

train <-read.csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/cancer/train.csv",header=T)
test <-read.csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/cancer/test.csv",header=T)
subdata <-read.csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/cancer/submission.csv",header=T)

summary(train)

#변수 스케일링
minmax<-preProcess(train,"range")
train$perimeter_se <- predict(minmax, train)$perimeter_se
train$area_se<- predict(minmax, train)$area_se
train$area_worst<- predict(minmax, train)$area_worst

train$diagnosis<-as.factor(train$diagnosis)
#결측치 제거
colSums(is.na(train))
#train,test 분리
idx <- sample(1:nrow(train), 0.7*nrow(train))
df_train <- train[idx,]
df_test <- train[-idx,]
str(train)
m <- randomForest(diagnosis~.-(id), data=df_train, ntree=100) #랜덤포레스트
p <- predict(m,newdata=df_test)

# F1 score
caret::confusionMatrix(df_test$diagnosis, p)$byClass[7]


# Accuracy
caret::confusionMatrix(df_test$diagnosis, p)$overall[1]

#ROC auc
auc(df_test$diagnosis, p) #랜덤포레스트


m2 <- randomForest(diagnosis~.-(id), data=train, ntree=100)
p2 <- predict(m, newdata=test,type="response")
length(p)
result<-as.data.frame(as.character(p))
# head(result)
write.csv(result, "diagnosis.csv", row.names=F)
abc<-read.csv("diagnosis.csv")
abc


